In [17]:
import pandas as pd
import numpy as np

In [2]:
df_reviews = pd.read_csv('./csv/reviews.csv')
df_reviews[:2]

,_id,co_date,co_review
0,65debc4d10cac7425905e881,2020년 2월 6일,"희귀 질환에 대한 정보나 관리가 어려워하시는 분들도 많으신데, 이런 어플은 정말 도..."
1,65debc5610cac7425905e883,2020년 2월 9일,디자인이 심플하고 직관적이라서 좋아요. 희귀질환에 대해 잘 모르고 있던 사람도 관심...


In [32]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는', '대해', '통해', '은', '이다', '이 ', '이제', '다시','아주','정말','대한','진짜']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [33]:
#tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2)

In [34]:
features = tfidfVectorizer.fit_transform(df_reviews["co_review"])

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [36]:
components_3  = LatentDirichletAllocation(n_components=3, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기
components_3.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=-1)

In [37]:
dictionary_list = tfidfVectorizer.get_feature_names_out(features)
dictionary_list

array(['가능성', '가입', '가장', '가족', '가지', '감사', '개발', '개선', '개인', '건강', '검사',
       '결과', '고생', '고통', '공유', '관련', '관리', '관심', '구성', '기능', '기대', '기분',
       '기타', '난치병', '내용', '노력', '노트', '누구', '다음', '당사자', '대상', '더욱',
       '데이터', '도움', '등록', '디자인', '루푸스', '매우', '발전', '번역', '보고', '보기',
       '부분', '사람', '사용', '사용자', '상세', '생각', '서로', '서비스', '선택', '소식', '스케',
       '신약', '심플', '아이디어', '어플', '업계', '업데이트', '여러', '우리', '위해', '유전자',
       '응원', '의료', '의학', '이용', '이용자', '이프', '일단', '자료', '자체', '적극', '접근',
       '정리', '정보', '제공', '좀더', '종류', '종사', '주변', '줄기', '증후군', '지금', '지속',
       '질병', '질환', '찾기', '최신', '추가', '추천', '출시', '취지', '치료', '컨텐츠', '크게',
       '파킨슨병', '평소', '포함', '항목', '항상', '해결', '해당', '해주시', '화이팅', '확대',
       '확인', '환자', '황반변성', '회원', '휴먼', '희귀', '희귀질환', '희망', '희망이'],
      dtype=object)

In [38]:
topics_output = components_3.transform(features)
df_topics_review = pd.DataFrame(data=topics_output)

In [39]:
df_topics_review['topic_num'] = np.argmax(topics_output, axis=1)
df_topics_review['review'] = df_reviews['co_review']
df_topics_review

,0,1,2,topic_num,review
0,0.259405,0.076604,0.663990,2,"희귀 질환에 대한 정보나 관리가 어려워하시는 분들도 많으신데, 이런 어플은 정말 도..."
1,0.481612,0.248334,0.270053,0,디자인이 심플하고 직관적이라서 좋아요. 희귀질환에 대해 잘 모르고 있던 사람도 관심...
2,0.545346,0.101000,0.353654,0,멜라스증후군에 대해서 공부하며 최근 이 앱을 알게된 학생입니다. 이 앱은 개인의 노...
3,0.576625,0.157340,0.266035,0,어려운 사용자를 위해 글씨크기도 크게 해주셨네요. 감사해요. 가끔 공지들 뜨면서 질...
4,0.427110,0.494116,0.078775,1,"아무래도 희귀질환이다보니 환자분들이 최신 의학 소식을 접하기가 어려우실텐데, 본인에..."
...,...,...,...,...,...
173,0.333333,0.333333,0.333333,0,ALL LIVES HAVE EQUAL VALUE
174,0.333333,0.333333,0.333333,0,즣음
175,0.333333,0.333333,0.333333,0,굿샷
176,0.333333,0.333333,0.333333,0,굿


In [40]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in components_3.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topics_keywords = pd.DataFrame(topics_list_add)

정보 도움 희귀질환 환자 희귀 디자인 관련 난치병 기능 생각
관심 소식 치료 보기 업데이트 결과 데이터 의료 의학 희귀질환
어플 질환 추가 사용 증후군 희귀 정리 추천 확인 상세


In [41]:
df_topics_keywords

,0,1,2
0,정보 도움 희귀질환 환자 희귀 디자인 관련 난치병 기능 생각,관심 소식 치료 보기 업데이트 결과 데이터 의료 의학 희귀질환,어플 질환 추가 사용 증후군 희귀 정리 추천 확인 상세
1,Topic0,Topic1,Topic2


In [42]:
pivot_table = pd.pivot_table(data=df_topics_review,
               values='review',
               index='topic_num',
               aggfunc='count')
pivot_table

,review
topic_num,
0,102
1,29
2,47


In [43]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [44]:
components_3  = LatentDirichletAllocation(n_components=3, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기
components_3.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=-1)

In [46]:
vis = pyLDAvis.lda_model.prepare(components_3, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [47]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소